In [1]:
import pandas as pd
import numpy as np

from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow .keras.utils import to_categorical

from sklearn.model_selection import train_test_split

In [2]:
import shfl

from shfl.data_base.data_base import LabeledDatabase
from shfl.private.reproducibility import Reproducibility

from shfl.differential_privacy import SensitivitySampler
from shfl.differential_privacy import L1SensitivityNorm

from shfl.differential_privacy import LaplaceMechanism

### 데이터 전처리

In [3]:
# 사전에 정의한 데이터 호출
df = pd.read_csv('./fd_data/test.csv')

In [4]:
# 전처리 1차 과정
encoder = preprocessing.LabelEncoder()

df['SCTR_CD'].fillna(0, inplace=True)
df['SCTR_CD'] = encoder.fit_transform(df['SCTR_CD'])

df['GENDER'].fillna(0, inplace=True)
df['GENDER'] = encoder.fit_transform(df['GENDER'])

df['LN_CD_1'].fillna(0, inplace=True)
df['LN_CD_1']= encoder.fit_transform(df['LN_CD_1'])

df['LN_CD_2'].fillna(1000, inplace=True)
df['LN_CD_2'] = encoder.fit_transform(df['LN_CD_2'])
df.replace({'LN_CD_2': {0: 20}}, inplace = True)
df.replace({'LN_CD_2': {19: 0}}, inplace = True)
df.replace({'LN_CD_2': {20: 19}}, inplace = True)

df['CD_OPN_CD_1'].fillna(0, inplace=True)
df['CD_OPN_CD_2'].fillna(0, inplace=True)
df['CD_OPN_CD_1'] = encoder.fit_transform(df['CD_OPN_CD_1'])
df['CD_OPN_CD_2'] = encoder.fit_transform(df['CD_OPN_CD_2'])

df['LN_AMT'].fillna(0, inplace=True)
df['DLQ_AMT'].fillna(0, inplace=True)
df['DLQ_TYPE'].fillna(0, inplace=True)
df['DLQ_CD_1'].fillna(0, inplace=True)

df['DLQ_CD_1'] = encoder.fit_transform(df['DLQ_CD_1'])
df['DLQ_CD_2'].fillna(10, inplace=True)
df['DLQ_CD_2'] = encoder.fit_transform(df['DLQ_CD_2'])

df.replace({'DLQ_CD_2': {0: 3}}, inplace = True)
df.replace({'DLQ_CD_2': {2: 0}}, inplace = True)
df.replace({'DLQ_CD_2': {3: 2}}, inplace = True)

In [5]:
# 전처리 2차 수정
df.drop("LN_YM",axis=1, inplace= True)
df.drop("DLQ_YM",axis=1, inplace= True)
df.drop("CD_OPN_YM",axis=1, inplace= True)
df.drop("LN_CD_1",axis=1, inplace= True)
df.drop("COM_KEY",axis=1, inplace= True)

df=df.loc[:,["YM","JOIN_KEY","BTH_YR","GENDER","SCTR_CD","LN_AMT","DLQ_TYPE","DLQ_CD_1","DLQ_CD_2","DLQ_AMT","CD_OPN_CD_1","CD_OPN_CD_2","LN_CD_2"]]

wanted_year = df['YM'] == '2016-12-01'
df = df[wanted_year]
df.BTH_YR = df.YM.str[:4].astype('int')-df.BTH_YR.str[:4].astype('int')
df.drop("YM",axis=1, inplace= True)
df= df.drop("JOIN_KEY", axis=1)

In [6]:
df

,BTH_YR,GENDER,SCTR_CD,LN_AMT,DLQ_TYPE,DLQ_CD_1,DLQ_CD_2,DLQ_AMT,CD_OPN_CD_1,CD_OPN_CD_2,LN_CD_2
0,26,0,1,19000.0,0.0,0,0,0.0,0,0,6
1,62,1,0,0.0,0.0,0,0,0.0,1,1,0
2,62,1,0,0.0,0.0,0,0,0.0,1,1,0
3,62,1,0,0.0,0.0,0,0,0.0,1,2,0
4,62,1,0,0.0,0.0,0,0,0.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6060,41,0,0,0.0,0.0,0,0,0.0,1,1,0
6061,79,0,0,0.0,0.0,0,0,0.0,1,1,0
6062,43,0,0,0.0,0.0,0,0,0.0,1,1,0
6063,36,0,0,0.0,0.0,0,0,0.0,1,1,0


In [7]:
df2= df

In [8]:
df2 = df2.rename({'BTH_YR':"나이","GENDER":"성별","SCTR_CD":"업권코드","LN_AMT":"대출금액","DLQ_TYPE":"연체유형코드","DLQ_CD_1":"연체사유코드","DLQ_CD_2":"등록사유코드","DLQ_AMT":"연체등록금액","CD_OPN_CD_1":"개설사유코드","CD_OPN_CD_2":"카드유형코드","LN_CD_2":"대출상품"},axis='columns')

In [9]:
df2.head()

,나이,성별,업권코드,대출금액,연체유형코드,연체사유코드,등록사유코드,연체등록금액,개설사유코드,카드유형코드,대출상품
0,26,0,1,19000.0,0.0,0,0,0.0,0,0,6
1,62,1,0,0.0,0.0,0,0,0.0,1,1,0
2,62,1,0,0.0,0.0,0,0,0.0,1,1,0
3,62,1,0,0.0,0.0,0,0,0.0,1,2,0
4,62,1,0,0.0,0.0,0,0,0.0,1,1,0


In [10]:
# 깔끔하게 정리해서 저장
df.to_csv('./fd_data/data.csv' , sep=',',encoding='utf-8-sig',index = False)

In [11]:
df

,BTH_YR,GENDER,SCTR_CD,LN_AMT,DLQ_TYPE,DLQ_CD_1,DLQ_CD_2,DLQ_AMT,CD_OPN_CD_1,CD_OPN_CD_2,LN_CD_2
0,26,0,1,19000.0,0.0,0,0,0.0,0,0,6
1,62,1,0,0.0,0.0,0,0,0.0,1,1,0
2,62,1,0,0.0,0.0,0,0,0.0,1,1,0
3,62,1,0,0.0,0.0,0,0,0.0,1,2,0
4,62,1,0,0.0,0.0,0,0,0.0,1,1,0
...,...,...,...,...,...,...,...,...,...,...,...
6060,41,0,0,0.0,0.0,0,0,0.0,1,1,0
6061,79,0,0,0.0,0.0,0,0,0.0,1,1,0
6062,43,0,0,0.0,0.0,0,0,0.0,1,1,0
6063,36,0,0,0.0,0.0,0,0,0.0,1,1,0


In [12]:
scaler = StandardScaler()
#df = scaler.fit_transform(df)

X = df.loc[:,['BTH_YR','GENDER','SCTR_CD','LN_AMT','DLQ_TYPE','DLQ_CD_1','DLQ_CD_2','DLQ_AMT','CD_OPN_CD_1','CD_OPN_CD_2']]
y = df.loc[:,['LN_CD_2']]

In [13]:
# Data와 label 나눠서 저장

X.to_csv('./fd_data/X.csv' , sep=',',encoding='utf-8-sig',index = False)
y.to_csv('./fd_data/y.csv' , sep=',',encoding='utf-8-sig',index = False)

In [14]:
scaler = scaler = StandardScaler()
X = scaler.fit_transform(X)
y = to_categorical(y)

In [15]:
trnx, tstx, trny, tsty = train_test_split(X, y, test_size=0.3)
trnx.shape, tstx.shape, trny.shape, tsty.shape

((4245, 10), (1820, 10), (4245, 20), (1820, 20))

### 일반 딥러닝 모델 적용

In [16]:
input_shape = (10,)

mlp_model = models.Sequential()
mlp_model.add(layers.Dense(units=1000, activation='relu', input_shape=input_shape))
mlp_model.add(layers.Dense(units=20, activation = 'softmax'))
mlp_model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

mlp_model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1000)              11000     
_________________________________________________________________
dense_1 (Dense)              (None, 20)                20020     
Total params: 31,020
Trainable params: 31,020
Non-trainable params: 0
_________________________________________________________________


In [17]:
history = mlp_model.fit(trnx, trny, validation_data = (tstx,tsty), batch_size=20, epochs=100, verbose=1)

Epoch 1/100
213/213 [==============================] - 2s 7ms/step - loss: 1.3077 - accuracy: 0.7022 - val_loss: 0.5951 - val_accuracy: 0.8275
Epoch 2/100
213/213 [==============================] - 0s 2ms/step - loss: 0.5426 - accuracy: 0.8424 - val_loss: 0.5654 - val_accuracy: 0.8308
Epoch 3/100
213/213 [==============================] - 0s 2ms/step - loss: 0.5286 - accuracy: 0.8427 - val_loss: 0.5344 - val_accuracy: 0.8324
Epoch 4/100
213/213 [==============================] - 0s 2ms/step - loss: 0.4777 - accuracy: 0.8458 - val_loss: 0.5364 - val_accuracy: 0.8412
Epoch 5/100
213/213 [==============================] - 0s 2ms/step - loss: 0.4980 - accuracy: 0.8418 - val_loss: 0.5286 - val_accuracy: 0.8390
Epoch 6/100
213/213 [==============================] - 0s 2ms/step - loss: 0.4842 - accuracy: 0.8454 - val_loss: 0.5205 - val_accuracy: 0.8423
Epoch 7/100
213/213 [==============================] - 0s 2ms/step - loss: 0.4553 - accuracy: 0.8541 - val_loss: 0.5171 - val_accuracy: 0.8462

Epoch 58/100
213/213 [==============================] - 0s 2ms/step - loss: 0.3141 - accuracy: 0.8932 - val_loss: 0.6670 - val_accuracy: 0.8593
Epoch 59/100
213/213 [==============================] - 0s 2ms/step - loss: 0.3181 - accuracy: 0.8834 - val_loss: 0.6734 - val_accuracy: 0.8533
Epoch 60/100
213/213 [==============================] - 0s 2ms/step - loss: 0.3085 - accuracy: 0.8902 - val_loss: 0.6630 - val_accuracy: 0.8626
Epoch 61/100
213/213 [==============================] - 0s 2ms/step - loss: 0.3104 - accuracy: 0.8921 - val_loss: 0.6816 - val_accuracy: 0.8566
Epoch 62/100
213/213 [==============================] - 0s 2ms/step - loss: 0.3152 - accuracy: 0.8836 - val_loss: 0.6719 - val_accuracy: 0.8577
Epoch 63/100
213/213 [==============================] - 0s 2ms/step - loss: 0.3184 - accuracy: 0.8866 - val_loss: 0.6632 - val_accuracy: 0.8599
Epoch 64/100
213/213 [==============================] - 0s 2ms/step - loss: 0.3037 - accuracy: 0.8941 - val_loss: 0.7021 - val_accuracy:

In [18]:
mlp_df =  pd.DataFrame(history.history, columns = ['loss', 'accuracy', 'val_loss', 'val_accuracy'])

mlp_df[0:100:10]

,loss,accuracy,val_loss,val_accuracy
0,0.858872,0.780683,0.595109,0.827473
10,0.429958,0.859128,0.514753,0.850000
20,0.385145,0.868787,0.546551,0.852747
30,0.366477,0.876090,0.580412,0.858791
40,0.351233,0.875854,0.605247,0.852198
50,0.337236,0.880565,0.635847,0.856044
60,0.329618,0.887397,0.681589,0.856593
70,0.320345,0.887397,0.699881,0.856593
80,0.312683,0.891637,0.762754,0.858242
90,0.305661,0.889988,0.814075,0.853297


### 연합학습 수행

In [19]:
database = LabeledDatabase(X, y)
train_data, train_labels, test_data, test_labels = database.load_data()

In [20]:
print("Shape of train_data: " + str(train_data.shape))
print("Shape of train_labels: " + str(train_labels.shape))
print("One sample features: " + str(train_data[0]))
print("One sample label: " + str(train_labels[0]))

Shape of train_data: (4852, 10)
Shape of train_labels: (4852, 20)
One sample features: [-1.39985943 -0.87090799  2.71963069 -0.2509778   3.16342528  4.91137005
  4.34905232 -0.02202424 -1.43932367 -1.43812271]
One sample label: [1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [21]:
import tensorflow as tf

def model_builder():
    # create model
    model = tf.keras.models.Sequential()
    model.add(layers.Dense(units=1000, activation='relu', input_shape=input_shape))
    model.add(layers.Dense(units=20, activation = 'softmax'))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return shfl.model.DeepLearningModel(model=model)

In [22]:
iid_distribution = shfl.data_distribution.IidDataDistribution(database)
federated_data, test_data, test_label = iid_distribution.get_federated_data(num_nodes=20, percent=10)

aggregator = shfl.federated_aggregator.FedAvgAggregator()
federated_government = shfl.federated_government.FederatedGovernment(model_builder, federated_data, aggregator)

federated_data, test_data.shape, test_label.shape

C:\Users\pmcsh\Anaconda3\envs\tensorflow\lib\site-packages\shfl\data_distribution\data_distribution_iid.py:67: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  federated_data = np.array(federated_data)
C:\Users\pmcsh\Anaconda3\envs\tensorflow\lib\site-packages\shfl\data_distribution\data_distribution_iid.py:68: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  federated_label = np.array(federated_label)


(<shfl.private.federated_operation.FederatedData at 0x23bb060c988>,
 (1213, 10),
 (1213, 20))

In [23]:
import numpy as np

class Reshape(shfl.private.FederatedTransformation):
    
    def apply(self, labeled_data):
        print(labeled_data.label.shape)
        #labeled_data.label = np.reshape(labeled_data.label, (labeled_data.label.shape[0], 1))
        
shfl.private.federated_operation.apply_federated_transformation(federated_data, Reshape())

(24, 20)
(24, 20)
(24, 20)
(24, 20)
(25, 20)
(24, 20)
(24, 20)
(24, 20)
(25, 20)
(24, 20)
(24, 20)
(24, 20)
(25, 20)
(24, 20)
(24, 20)
(25, 20)
(24, 20)
(24, 20)
(24, 20)
(25, 20)


In [ ]:
#test_label = np.reshape(test_label, (test_label.shape[0], 1))
federated_government.run_rounds(200, test_data, test_label)

Accuracy round 0


Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060C848>: [2.9450623989105225, 0.029678482562303543]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CC08>: [2.95094895362854, 0.03132728859782219]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CD48>: [2.936021566390991, 0.02885408140718937]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CE88>: [2.9539051055908203, 0.0387469083070755]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CFC8>: [2.9440927505493164, 0.023083264008164406]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602188>: [2.945133686065674, 0.12448474764823914]


Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB06022C8>: [2.948089122772217, 0.035449299961328506]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602408>: [2.9513559341430664, 0.05605935677886009]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602548>: [2.9474239349365234, 0.02390766702592373]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602648>: [2.944025754928589, 0.05605935677886009]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602788>: [2.9402413368225098, 0.02638087421655655]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0615A88>: [2.9599382877349854, 0.019785655662417412]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB054DCC8>:

C:\Users\pmcsh\Anaconda3\envs\tensorflow\lib\site-packages\shfl\federated_aggregator\fedavg_aggregator.py:26: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  clients_params_array = np.array(clients_params)
C:\Users\pmcsh\Anaconda3\envs\tensorflow\lib\site-packages\shfl\federated_aggregator\fedavg_aggregator.py:29: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray
  aggregated_weights = np.array([np.mean(clients_params_array[:, layer], axis=0) for layer in range(shape)])


Global model test performance : [2.9465842247009277, 0.025556471198797226]



Accuracy round 1
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060C848>: [2.8455862998962402, 0.583676815032959]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CC08>: [2.8512930870056152, 0.5721352100372314]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CD48>: [2.836334466934204, 0.5886232256889343]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CE88>: [2.854337692260742, 0.5605935454368591]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CFC8>: [2.8441717624664307, 0.5432811379432678]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602188>: [2.8453919887542725, 0.5581203699111938]
Test performance cli

...

Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB054DCC8>: [0.7907137870788574, 0.7980214357376099]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0535988>: [0.7946217656135559, 0.7996702194213867]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0535248>: [0.8015753626823425, 0.7947238087654114]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAF967848>: [0.7895422577857971, 0.7914262413978577]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAF967D08>: [0.7928484678268433, 0.7955482006072998]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAF967A48>: [0.8028247356414795, 0.7922506332397461]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB05567C8>: [0.80

Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB06022C8>: [0.8025223612785339, 0.7955482006072998]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602408>: [0.7947543263435364, 0.7930750250816345]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602548>: [0.7957496643066406, 0.796372652053833]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602648>: [0.7945442795753479, 0.7980214357376099]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602788>: [0.7967802286148071, 0.7922506332397461]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0615A88>: [0.7892495393753052, 0.7988458275794983]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB054DCC8>: [0.787

Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060C848>: [0.7893975377082825, 0.7930750250816345]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CC08>: [0.7829565405845642, 0.796372652053833]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CD48>: [0.7959577441215515, 0.7955482006072998]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CE88>: [0.7886837124824524, 0.7947238087654114]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CFC8>: [0.7877904176712036, 0.796372652053833]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602188>: [0.7926831245422363, 0.7947238087654114]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB06022C8>: [0.7993

Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAF967D08>: [0.7842868566513062, 0.796372652053833]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAF967A48>: [0.7945049405097961, 0.7922506332397461]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB05567C8>: [0.7930253148078918, 0.7947238087654114]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAFD40DC8>: [0.7836991548538208, 0.8004946708679199]
Global model test performance : [0.7870827317237854, 0.796372652053833]



Accuracy round 100
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060C848>: [0.7864607572555542, 0.793899416923523]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB060CC08>: [0.779946506023407, 0.796372652053833]
Test performance clien

Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0602788>: [0.7893289923667908, 0.793899416923523]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0615A88>: [0.7814443111419678, 0.8013190627098083]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB054DCC8>: [0.7799935936927795, 0.7996702194213867]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0535988>: [0.7838649749755859, 0.7988458275794983]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BB0535248>: [0.7905362248420715, 0.7947238087654114]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAF967848>: [0.7777345776557922, 0.7955482006072998]
Test performance client <shfl.private.federated_operation.FederatedDataNode object at 0x0000023BAF967D08>: [0.781

### 연합학습 + 차등프라이버시

In [ ]:
# 오류가 뜨기 때문에 임의로 지정

max_sensitivity = 0.21304818987846375
mean_sensitivity = 0.019892680879049707

In [ ]:
params_access_definition = LaplaceMechanism(sensitivity=mean_sensitivity, epsilon=0.5)
federated_governmentDP = shfl.federated_government.FederatedGovernment(
    model_builder, federated_data, aggregator, model_params_access=params_access_definition)
federated_governmentDP.run_rounds(100, test_data, test_label)

In [ ]:
params_access_definition = LaplaceMechanism(sensitivity=mean_sensitivity, epsilon=0.5)
federated_governmentDP = shfl.federated_government.FederatedGovernment(
    model_builder, federated_data, aggregator, model_params_access=params_access_definition)
federated_governmentDP.run_rounds(200, test_data, test_label)

### 제외 문장

In [ ]:
# 오류가 뜨는 문장

class UniformDistribution(shfl.differential_privacy.ProbabilityDistribution):
    """
    Implement Uniform sampling over the data
    """
    def __init__(self, sample_data):
        self._sample_data = sample_data

    def sample(self, sample_size):
        row_indices = np.random.randint(low=0, high=self._sample_data.shape[0], size=sample_size, dtype='l')
        
        return self._sample_data[row_indices, :]
    

class DeepLearningSample(shfl.model.DeepLearningModel):
    """
    Adds the "get" method to model's class
    """
    def get(self, data_array):
        data = data_array[:, 0:-1]
        labels = data_array[:, -1].reshape(-1,1)
        train_model = self.train(data, labels)
        
        return self.get_model_params()


def model_builder_sample():
    # create model
    model = tf.keras.models.Sequential()
    model.add(layers.Dense(units=1000, activation='relu', input_shape=input_shape))
    model.add(layers.Dense(units=20, activation = 'softmax'))
    model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return DeepLearningSample(model=model)


class L1SensitivityNormLists(L1SensitivityNorm):
    """
    Implements the L1 norm of the difference between lists of parameters x_1 and x_2
    """
    def compute(self, x_1, x_2):
        x = []
        for x_1_i, x_2_i in zip(x_1, x_2):
            x.append(np.sum(np.abs(x_1_i - x_2_i)))   
        
        return np.max(x) # This could be allowed to be an array

    
#sample_data = np.hstack((train_data, train_labels.reshape(-1,1)))
sample_data = np.hstack((train_data, train_labels))
distribution = UniformDistribution(sample_data)
sampler = SensitivitySampler()
n_samples = 100

max_sensitivity, mean_sensitivity = sampler.sample_sensitivity(
    model_builder_sample(), 
    L1SensitivityNormLists(), distribution, n=n_samples, gamma=0.05)

print("Max sensitivity from sampling: " + str(max_sensitivity))
print("Mean sensitivity from sampling: " + str(mean_sensitivity))